In [ ]:
from Crypto.Random.random import randint, choice
import os

# In a realm where curiosity roams free, our fearless cat sets out on an epic journey.
# Even the cleverest feline must respect the boundaries of its world—this magical limit holds all wonders within.
limit = 0xe5db6a6d765b1ba6e727aa7a87a792c49bb9ddeb2bad999f5ea04f047255d5a72e193a7d58aa8ef619b0262de6d25651085842fd9c385fa4f1032c305f44b8a4f92b16c8115d0595cebfccc1c655ca20db597ff1f01e0db70b9073fbaa1ae5e489484c7a45c215ea02db3c77f1865e1e8597cb0b0af3241cd8214bd5b5c1491f
# 161411139670770968531535247674401613102839567845219955087163205911090766193199881375203884967436075676810510094624490873765685331101041092890302638149771122520786415161307461143451707449979558081493648727968357526510996395524371672181873315066779647452231037655908300322299261984462076469824977852124652194079f

##### ***1. walking(x, y, part)***
- Biến đổi tuyến tính, `part` được chia thành 04 phần, mỗi phần có 2 byte (16bit)
- `epart` là danh sách 4 số `[a, b, c, d]` từ `part`
- Chuyển đổi `x`, `y` thành `xx`, `yy` theo công thức
$$xx = a.x+b.y$$
$$yy = c.x+d.y$$

In [ ]:
# Hàm biến đổi x, y thành xx, yy theo tuyến tính
def walking(x, y, part):
    # Each step is guided by a fragment of the cat's own secret mind.
    epart = [int.from_bytes(part[i:i+2], "big") for i in range(0, len(part), 2)]
    xx = epart[0] * x + epart[1] * y
    yy = epart[2] * x + epart[3] * y
    return xx, yy

#### **2. Lớp `Cat`**

##### ***2.1. `__init__(self)`***
- Khởi tạo một con mèo với vị trí ban đầu là số nguyên 256 bit ngẫu nhiên, `(x, y)` là vị trí của con mèo.
- Vòng lặp vô hạn, với `mind` là chuỗi 1000 byte ngẫu nhiên từ hệ thống, mỗi lần lặp sẽ cắt `mind` thành các đoạn nhỏ 8 byte ==> có 125 bước di chuyển. Kiểm tra tất cả các bước có khác nhau không? Mục tiêu là đảm bảo mỗi bước là duy nhất, khác nhau đôi một. 
- `self.step` lưu trữ các bước di chuyển đôi một khác nhau này.

##### ***2.2. `moving(self)`***
- Phương thức di chuyển của con mèo, tối đa 30 bước.
- Tại mỗi bước, con mèo chọn một `part` ngẫu nhiên từ `self.step`, sau đó biến đổi vị trí `(x, y)` --> `(xx, yy)`.
- Nếu vượt khỏi giới hạn (`limit`), thì modulo trở lại vùng hợp lệ và dừng lại.

##### ***2.3. `position(self)`***
- Lấy vị trí hiện tại của con mèo.

In [ ]:
class Cat:
    def __init__(self):
        # The cat's starting position is born of pure randomness.
        self.x = randint(0, 2**256)
        self.y = randint(0, 2**256)
        # Deep within, its mind holds a thousand mysterious fragments.
        while True:
            self.mind = os.urandom(1000)
            self.step = [self.mind[i:i+8] for i in range(0, 1000, 8)]
            if len(set(self.step)) == len(self.step):
                break

    # The epic chase begins: the cat ponders and strides toward the horizon.
    def moving(self):
        for _ in range(30):
            # A moment of reflection: choose a thought from the cat's endless mind.
            part = choice(self.step)
            self.step.remove(part)
            # With each heartbeat, the cat takes a cryptic step.
            xx, yy = walking(self.x, self.y, part)
            self.x, self.y = xx, yy
            # When the wild spirit reaches the edge, it respects the boundary and pauses.
            if self.x > limit or self.y > limit:
                self.x %= limit
                self.y %= limit
                break

    # When the cosmos beckons, the cat reveals its secret coordinates.
    def position(self):
        return (self.x, self.y)

#### **3. Bắt đầu thử thách**

- Thử thách bao gồm 20 vòng, mỗi vòng yêu cầu tìm một con mèo.
- Khởi tạo con mèo, đưa ra vị trí con mèo lúc ban đầu. In ra tất cả các bước di chuyển của con mèo (125 bước)
- Mèo thực hiện di chuyển (với 30 bước), sau đó in ra vị trí hiện tại của con mèo. 
- Yêu cầu: Người chơi tìm ra, dự đoán hoặc tính toán lại chính xác đường đi của mèo. Nhập một chuỗi hex, sẽ được chia lại thành từng đoạn 8 byte. 
- Nói cách khác: Người chơi cần tính toán lại 30 bước đi của con mèo.

In [ ]:
for round in range(20):
    try:
        print(f"👉 Hunt {round+1}/20 begins!")
        cat = Cat()

        # At the start, you and the cat share the same starlit square.
        human_pos = cat.position()
        print(f"🐱✨ Co-location: {human_pos}")
        print(f"🔮 Cat's hidden mind: {cat.mind.hex()}")

        # But the cat, ever playful, dashes into the unknown...
        cat.moving()
        print("😸 The chase is on!")

        print(f"🗺️ Cat now at: {cat.position()}")

        # Your turn: recall the cat's secret path fragments to catch up.
        mind = bytes.fromhex(input("🤔 Path to recall (hex): "))

        # Step by step, follow the trail the cat has laid.
        for i in range(0, len(mind), 8):
            part = mind[i:i+8]
            if part not in cat.mind:
                print("❌ Lost in the labyrinth of thoughts.")
                exit()
            human_pos = walking(human_pos[0], human_pos[1], part)

        # At last, if destiny aligns...
        if human_pos == cat.position():
            print("🎉 Reunion! You have found your feline friend! 🐾")
        else:
            print("😿 The path eludes you... Your heart aches.")
            exit()
    except Exception:
        print("🙀 A puzzle too tangled for tonight. Rest well.")
        exit()

# Triumph at last: the final cat yields the secret prize.
print(f"🏆 Victory! The treasure lies within: {open('flag.txt').read()}")